Remote Package Installation

In [1]:
!pip install torch
!pip install torchvision
!pip install numpy
!pip install torchmetrics
!pip install torchinfo
!pip install matplotlib
!pip install scikit-learn
!pip install pandas
!pip install matplotlib.pyplot
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 58.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.8 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 87.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 30.5 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.1-py3-none-any.whl size=136905 sha256=ac8eb66bfd43dafe55acb8e1d28e622b3b994ffa

Imports

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

import torchvision
import torchvision.models as models
from torchvision import *
from torchvision import datasets
from torchvision.models.feature_extraction import create_feature_extractor

from torchmetrics import *

from torch.utils.data import *

import sklearn as sk
from sklearn import *
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

from itertools import compress


import matplotlib.pyplot as plt


import yfinance as yf


Data Processing

In [3]:
# time step to predict ahead, creates input sequences
# gonna have to figure out test
def create_lstm_data(data, time_step=1, future = 0,test_percent = .1):
    x_vec, y_vec = [], []
    # formats data so y = t and x = t-1, ... , t-time_steps
    for i in range(len(data) - time_step):
        split = i + time_step
        # checks if data is of same length for concatentation
        length = data[split : split + future, 0].shape[0]
        if(length == future):
            x_vec.append(data[i : split, 0].unsqueeze(0))
            y_vec.append(data[split : split + future, 0].unsqueeze(0))
    # calculate number of elements to allocate to test
    #dataset_length = len(x_vec)
    #num_of_train = dataset_length - (int)(test_percent * dataset_length)
    # concats x into matrix and y into vector, needs unsqueez to add single dimension for LSTM
    return torch.cat(x_vec,0).unsqueeze(-1), torch.cat(y_vec,0).unsqueeze(-1)#, torch.cat(x_vec[num_of_train:],0).unsqueeze(-1), torch.cat(y_vec[num_of_train:],0).unsqueeze(-1)

# create a dataset out of timeseries data, must be formatted first, tensor
# correct timeseries formatation
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,i):
        return self.X[i], self.y[i]
    

Model Architecture

In [161]:
#possibly change how layers of encoder to decoder are handled (possibly make different sizes)
#maybe bidirectional? i dont thinks so
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout = 0.5):
        super(Encoder,self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.input_dim = input_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=dropout,
                           batch_first=True, bidirectional = True)
        
        #to choose part of birdectinals were important
        self.fc_hidden = nn.Linear(hidden_dim*2, hidden_dim)
        self.fc_cell = nn.Linear(hidden_dim*2, hidden_dim)

    def forward(self, x):
        
        outputs, (hidden, cell) = self.rnn(x)
        #print(x.shape)
        print(hidden.view(-1,64,self.hidden_dim).shape)
        

        #chooses which way of directionsal is most important
        hidden = self.fc_hidden(torch.cat((hidden[0:1],hidden[1:2]), dim = 2))
        cell = self.fc_cell(torch.cat((cell[0:1],cell[1:2]), dim = 2))
        #print(hidden.shape)
        return hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, dropout = 0.5):
        super(Decoder,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=dropout,
                           )
        self.fc_out = nn.Linear(hidden_dim, 1)

    #predicts a single time step, with either y pred as input or y
    def forward(self, y, prev_hidden, prev_cell):

        y = y.unsqueeze(0)

        output, (hidden, cell) = self.rnn(y, (prev_hidden, prev_cell))

        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell
    
class EncoderDecoderWrapper(nn.Module):
    def __init__(self, encoder, decoder, output_size = 1, teacher_forcing=0.3, device = 'cpu'):
        super(EncoderDecoderWrapper,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.teacher_forcing = teacher_forcing
        self.device = device
        self.output_size = output_size
        # might change if layers arent the same proves different results
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        )
        assert (
            encoder.n_layers == decoder.n_layers
        )
    def forward(self, source, target=None):
        
        # num of elements in each batch
        batch_size = source.shape[0]
        #should be same size as output
        
        if(target != None):
            target_len = target.shape[1]
        
            assert(target_len == self.output_size)
        
        prev_hidden, prev_cell = self.encoder(source)

        prev_target = source[:,-1]
            
        outputs = torch.zeros(batch_size,self.output_size).to(self.device)
        
        for t in range(self.output_size):
            
            prediction, prev_hidden, prev_cell = self.decoder(prev_target, prev_hidden, prev_cell)

            outputs[:,t] = prediction.squeeze(1)
            
            #chance of using actual vs chance of using predicted in training
            prev_target = target[:,t] if torch.rand(1) < self.teacher_forcing or target != None else prediction
        
        return outputs

Training Loop

In [56]:
def train_epoch(model,dataloader,loss_fn,optimizer,device = 'cpu'):
    
    model.train()
    running_loss = 0.0

    for X, y in dataloader:
        X_train, y_train = X.to(device).to(torch.float32), y.to(device).to(torch.float32)
        
        output = model(X_train, y_train)
        loss = loss_fn(output, y_train.squeeze(2))
        running_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_loss = running_loss / len(dataloader)
    print(f'Train Average Loss: {avg_loss}')

Formatting Data

In [57]:
# Download stock data from Yahoo Finance
hmc_data = yf.download('HMC', period='10y')['Close']
hymtf_data = yf.download('HYMTF', period='10y')['Close']
# get spread
spread = hmc_data - hymtf_data
spread = spread.values.reshape(-1,1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [58]:
#parameters for data
BATCH_SIZE = 64
input_size = 30
output_size = 60

In [59]:
scaler = MinMaxScaler(feature_range=(0,1))
#transforms/normalizes data and converts it to a tensor
normalized_data = torch.tensor(scaler.fit_transform(spread))
X_train, y_train = create_lstm_data(normalized_data, input_size, output_size)
#creates dataset
train_dataset = TimeSeriesDataset(X_train,y_train)
#test_dataset = TimeSeriesDataset(X_test,y_test)
#puts dataset into dataloader
train_dataloader = DataLoader(dataset = train_dataset, batch_size = BATCH_SIZE)
#test_dataloader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE)

Training

In [158]:
#hyper parameters
input_dim = 1
num_layers = 1
hidden_size = 16
dropout = 0.5
teacher_forcing = 0.2


device = device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoder = Encoder(input_dim,num_layers,hidden_size,dropout)
decoder = Decoder(input_dim,num_layers,hidden_size,dropout)
model = EncoderDecoderWrapper(encoder,decoder,output_size,teacher_forcing,device).to(device)

In [159]:
lr = 0.001
num_epochs = 100
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr)

In [160]:
for epoch in range(num_epochs):
    print(f'Epoch: {epoch}')
    train_epoch(model, train_dataloader, loss, optimizer,device)

Epoch: 0
torch.Size([64, 30, 1])
torch.Size([32, 64, 1])
torch.Size([1, 64, 1])


RuntimeError: Expected hidden[0] size (16, 64, 1), got [1, 64, 1]

Testing

In [ ]:
with torch.inference_mode():
    predicted = model(X_train.to(device).to(torch.float32),y_train.to(device).to(torch.float32)).to(device)

#y_train = format_back(y_train, 7)
#y_pred = format_back(predicted, 7)

print(f'{predicted.shape} | {X_train.shape} | {y_train.shape}')

i = 300

pred_with_x = torch.concat([X_train.cpu().squeeze(2),predicted.cpu()],1)
y_with_x = torch.concat([X_train.cpu().squeeze(2),y_train.cpu().squeeze(2)],1)

true_y_train = scaler.inverse_transform(pred_with_x)
true_y_pred = scaler.inverse_transform(y_with_x)

plt.plot(true_y_train[i], label = 'Actual Close')
plt.plot(true_y_pred[i], label = 'Predicted Close')
plt.legend()

#true_y_pred[1], true_y_train[1]